## Iterables and Generators

If our data doesn't fit in memory, we need to fetch  the items lazily- one at a time and on demand.

Iterable is an object that provides iterator and iterator is used for  the following tasks:
- For loops.
- List, dict set comprehension.
- Unpacking assignments.
- Construction of collection instances.


Whenever python wants to iterate over an object, it first looks for `__iter__` function then looks for `__getitem__` function and if neither is implemented then it raises a TypeError.



#### Duck typing
>"If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck."


In programming, duck typing means that the type or class of an object is determined by its behavior (methods and properties) rather than its explicit type or inheritance hierarchy.

Duck typing is used in iterables meaning that if the object implements the `__getitem__` then it is iterable without having  `__iter__` function.


#### Example 1

the first takeaway is that iterable is the object that implements either of `__iter__` or `__getitem__`.


In [3]:
from collections import abc

class duck:
    def __getitem__(self, i):
        pass

class goose:
    def __iter__(self):
        pass

print(issubclass(duck, abc.Iterable))
print(issubclass(goose, abc.Iterable))

False
True



but still you can iterate both of them. --> duck typing!


#### Example 2

We use iter along with a callable and a sentinel which is the the value that stops the iteration.

Key takeaway here is the iter function which can get another callable and a sentinel and return an Iterator

In [3]:
import random

def pick_a_name():
    names = ["Ali", "Atiyeh", "Hasssan", "Zahra", "Maryam", "Leila", "STOP"]
    index  = random.randint(0, len(names)-1)
    return names[index]


names_iter = iter(pick_a_name, "STOP")
print(names_iter)
for name in names_iter:
    print(name)

Atiyeh
Zahra
Zahra
Hasssan
Zahra
Leila
Hasssan
Maryam
Atiyeh
Hasssan
Zahra
Atiyeh
Zahra
Atiyeh
Atiyeh
Leila
Atiyeh
Leila
Hasssan


#### Example 3:
 point: Once the iterator is exhausted, you should define a new one.

 when we use `list()` it exhausts the iterator so then it gives the `StopIteration` and returns empty list

In [6]:

l = [1, 2, 3, 4, 5, 6, 7, 8]
it = iter(l)
for _ in range(len(l)-3):
    print(next(it))

print(list(it))

1
2
3
4
5
[6, 7, 8]


In [8]:
print(next(it))

StopIteration: 

iterables have an `__iter__` method that instantiates a new iterator everytime. Iterators implement a `__next__` method that returns individual items, and an `__iter__` method that returns self.

### Iterable vs. Iterator

The difference of iterable and iterator:


An **iterable** is any Python object capable of returning its elements one at a time.


An **iterator** is an object that represents a stream of data. It implements the `__next__()` method to fetch the next item in the sequence and raises StopIteration when no more items are left.





A lazy implementation of sentence class which yields items one by one and doesn't make a list of words.
finall is an eager valuation of re expressions while finditer is a lazy one which yields the patterns at runtime.

In [2]:
import re 
import reprlib 

RE_word = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text

    def __repr__(self) -> str:
        return 'Sentence(%s)'%reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_word.finditer(self.text):
            yield match.group()

so we have different ways to define a lazy generator and one is to use a `genexp` and the other is `generator functions`.


`genexp` is a shortcut that without calling a function makes a generator. on the other hand, defining `generator functions` has more flexibility and can have complex logic and mulitple statements.

In [ ]:
import re 
import reprlib 

RE_word = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text

    def __repr__(self) -> str:
        return 'Sentence(%s)'%reprlib.repr(self.text)

    def __iter__(self):
        ## still a generator which returns the genexp
        return (match.group() for match in RE_word.finditer(self.text))

#### Difference of generator and iterator

It seems that generator is a subclass of Iterator.


|                    Iterator                     |                       Generator                          |
|-------------------------------------------------|----------------------------------------------------------|
|any object that implements `__next__` method.    | An iterator which is built by python compiler.           |
|Designed to produce data that is consumed by client code. |                                                 |
|Most iterators we use in python are generators.  |                                                          |
|Created by defining a class having the `__next__` method.| Created either by a `yield` keyword or `genexp`s.|

In [6]:
def foo():
    yield "hello"

gen = (x for x in range(10))

type(foo()), type(gen)

(generator, generator)

#### Question1

Implement a class SkipIterator that takes an iterable as input and skips every n-th item while iterating over it. You should:

- Implement it first using an Iterator class.

- Then implement the same functionality using a generator function.

In [25]:
class SkipIterator:
    def __init__(self, iterable, n):
        self.iterable = iterable
        self.n = n
        self.counter = -1

    def __iter__(self):
        return self

    def __next__(self):
        if self.counter >= len(self.iterable):
            raise StopIteration
        self.counter += 1
        if (self.counter + 1) % self.n == 0:
            self.counter += 1
            if self.counter >= len(self.iterable):
                raise StopIteration
        return self.iterable[self.counter]

In [27]:
## generator function
def SkipIterator_func(iterable, n):
    for i in range(len(iterable)):
        if (i+1) % n == 0:
            continue
        yield iterable[i]

In [28]:
s = SkipIterator([10, 20, 30, 40, 50, 60], 3)
for item in s:
    print(item)


10
20
40
50


In [29]:
s = SkipIterator_func([10, 20, 30, 40, 50, 60], 3)
for item in s:
    print(item)

10
20
40
50


#### Exmaples of ready-to-use generator functions

when implementing generators, know what is available in the standard library, otherwise there’s a good chance you’ll reinvent the wheel


In [2]:
import itertools

gen = itertools.count(1, .5)  ## returns a generator that yields numbers ##CAUTION:itertools.count never stops so dont call it with list

next(gen)

1

In [24]:
gen = itertools.takewhile(lambda n: n<10, itertools.count(1, 2))
list(gen)

[1, 3, 5, 7, 9]

In [14]:
def vowel(c):
    return c.lower() in 'aioue'

# print(list(itertools.filterfalse(vowel, "hello world")))
print(list(itertools.filterfalse(vowel, "Hello world")))
print(list(filter(vowel, 'Hello world')))

['H', 'l', 'l', ' ', 'w', 'r', 'l', 'd']
['e', 'o', 'o']


for takewhile and dropwhile the difference is that it iters until the first true predicate. it doesnt work like a filter it stops when the condition is True.

In [20]:
print(list(itertools.takewhile(vowel, 'aiouehello world')))
print(list(itertools.takewhile(vowel, 'hello world')))

['a', 'i', 'o', 'u', 'e']
[]


In [21]:
print(list(itertools.dropwhile(vowel, 'aiouehello world')))
print(list(itertools.dropwhile(vowel, 'hello world')))

['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd']
['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd']


In [25]:
print(list(itertools.compress("hello", [1,0,1,1,0])))

['h', 'l', 'l']


In [30]:
sample = [1,2,3,4,5,6,7,8,9,10]

print(list(itertools.accumulate(sample)))

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


In [34]:
list(itertools.accumulate(sample, min))

list(itertools.accumulate(sample, max))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [35]:
import operator
sample2 = [('a', 1), ('b', 2), ('c', 3)]

list(itertools.starmap(operator.mul, sample2))

['a', 'bb', 'ccc']

In [37]:
coords = list(map(lambda x, y: (x,y), [36, 42, 23], [32, 45, 67]))
coords

[(36, 32), (42, 45), (23, 67)]

In [38]:
text = 'Hello world'

list(enumerate(text, start=1))

[(1, 'H'),
 (2, 'e'),
 (3, 'l'),
 (4, 'l'),
 (5, 'o'),
 (6, ' '),
 (7, 'w'),
 (8, 'o'),
 (9, 'r'),
 (10, 'l'),
 (11, 'd')]

In [39]:
## concatenates the two iterables
list(itertools.chain("ABC", range(10)))

['A', 'B', 'C', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [40]:
list(itertools.zip_longest('ABC',[10, 20, 30 , 40, 50], fillvalue='_'))

[('A', 10), ('B', 20), ('C', 30), ('_', 40), ('_', 50)]

In [41]:
list(itertools.product('ABC', [1, 2, 3]))

[('A', 1),
 ('A', 2),
 ('A', 3),
 ('B', 1),
 ('B', 2),
 ('B', 3),
 ('C', 1),
 ('C', 2),
 ('C', 3)]

In [42]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [44]:
names = ['ali', 'zahra', 'maryam', 'atiyeh', 'hassan', 'reza', 'mojtaba']

for length, group in itertools.groupby(names, len):
    print(length, " -> ", list(group))

3  ->  ['ali']
5  ->  ['zahra']
6  ->  ['maryam', 'atiyeh', 'hassan']
4  ->  ['reza']
7  ->  ['mojtaba']


In [46]:
## it is also a generator
reversed(sample)

#### `yield from`

used when a generator needs to yield values from another generator.

Note that yield from pauses gen, and sub_gen takes over until it is exhausted.

| **`return`**                              | **`yield`**                                                           |
| ----------------------------------------- | --------------------------------------------------------------------- |
| **Ends** the function completely.         | **Pauses** the function, saving its state.                            |
| Returns a **single value** to the caller. | Returns a **generator object** that can produce a sequence of values. |
| Cannot resume from where it left.         | Can **resume execution from where it paused** on the next iteration.  |
| Used in **normal functions**.             | Used in **generator functions**.                                      |
| Example:                                  |                                                                       |


In [48]:
def sub_gen():
    yield 1.1
    yield 1.2

def gen():
    yield 1
    yield from sub_gen()
    yield 2


for i in gen():
    print(i)

1
1.1
1.2
2


In [49]:
def chain(*iterables):
    for it in iterables:
        yield from it


s = 'ABC'
r = range(3)

list(chain(s, r))

['A', 'B', 'C', 0, 1, 2]

In [55]:
def tree(cls, level=0):
    yield cls.__name__, level
    for sub_class in cls.__subclasses__():
        yield from tree(sub_class, level+1)

def display(cls):
    for cls_name, level  in tree(cls):
        indent = ' '* 4 * level
        print(f'{indent}{cls_name}')


if __name__=='__main__':
    display(BaseException)

BaseException
    BaseExceptionGroup
        ExceptionGroup
    Exception
        ArithmeticError
            FloatingPointError
            OverflowError
            ZeroDivisionError
                DivisionByZero
                DivisionUndefined
            DecimalException
                Clamped
                Rounded
                    Underflow
                    Overflow
                Inexact
                    Underflow
                    Overflow
                Subnormal
                    Underflow
                DivisionByZero
                FloatOperation
                InvalidOperation
                    ConversionSyntax
                    DivisionImpossible
                    DivisionUndefined
                    InvalidContext
        AssertionError
        AttributeError
            FrozenInstanceError
        BufferError
        EOFError
            IncompleteReadError
        ImportError
            ModuleNotFoundError
                PackageNotFoundE

#### Classic Coroutines

Coroutine -> a routine or function that can be stopped at some point and start again.

generators are commonly used as iterators, but they can be used as coroutines.

- Generators **produce** data for iteration.
- Coroutines are **consumers** of data.
- Coroutines are not related to iterators.

In [59]:
def foo():
    x = yield 
    print("Recieved:", x)

f = foo()
next(f)
f.send('hello')

Recieved: hello


StopIteration: 

In [69]:
from collections.abc import Generator

def averager() -> Generator[float, float, None]:
    total = 0.0
    count = 0.0
    average = 0.0
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg = averager()
coro_avg.send(None) ## or next(coro_avg)


0.0

In [70]:
coro_avg.send(30)

30.0

In [71]:
coro_avg.send(40)

35.0

In [72]:
## Returning a value from a coroutine

from collections.abc import Generator
from typing import Union, NamedTuple

class Result(NamedTuple):
    count : int
    average: float

class Sentinel:
    def __repr__(self):
        return f'<Sentinel>'
    
STOP = Sentinel()


def averager2(verbose:bool = False) -> Generator[None, float | Sentinel, Result]:
    total = 0.0
    count = 0.0
    average = 0.0
    while True:
        term = yield
        if verbose:
            print('Recieved: ', term)
        if isinstance(term, Sentinel):
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [75]:
avg = averager2()
next(avg)
avg.send(10)
avg.send(30)
avg.send(6.5)
try:
    avg.send(STOP)
    ## the StopIteration instance has a value attribute bound to the value of return
except StopIteration as exc:
    result = exc.value

print(result)

Result(count=3.0, average=15.5)


## Homeworks

Create a generator that generates the squares of numbers up to some number N.

In [76]:
def gen(N):
    for i in range(N):
        yield i**2

In [6]:
def classic_coroutine():
    value = yield "Ready to receive"
    print(value)
    while True:
        print(f"Received: {value}")
        value = yield f"Processed {value}"
        print(value)

In [7]:
coro = classic_coroutine()
print(next(coro))

Ready to receive


In [8]:
coro.send(42)

42
Received: 42


'Processed 42'

In [5]:
coro.send(10)

Received: 10


'Processed 10'

In [12]:
def test():
    yield 1
    print("1 is Done")
    yield 2
    print("2 is Done")
    return 3

g = test()
print(next(g))
print("---------")
print(next(g))
print("---------")
print(next(g))
print("---------")


1
---------
1 is Done
2
---------
2 is Done


StopIteration: 3

StopIteration: 